# Extracting Argument Premises Using Zero-Shot Prompting

This notebook applies zero-shot prompting to extract argument premises from the data entries.
There are two prompts: zero-shot and zero-shot chain-of-thought.

In [2]:
import os
import json
from tqdm import tqdm
from OllamaCached import llm_chat

In [3]:
# setting environment variables for OllamaCached model
os.environ["OLLAMA_MODELS"] = "./models"
os.environ["OLLAMA_KEEP_ALIVE"] = "1m"

In [4]:
# list of policy options associated with the data entries
policy_option_list = ['The municipality takes the lead and unburdens you',
                      'Inhabitants do it themselves',
                      'The market determines what is coming',
                      'Large-scale energy generation will occur in a small number of places',
                      'Betting on storage (Sudwest-Fryslan becomes the battery of the Netherlands)',
                      'Become a major energy supplier in the Netherlands']

In [5]:
annotator_number = 1
file_path = f'data_extraction/personal_annotation_{annotator_number}.json'

# load data
with open(file_path, 'r') as json_file:
    data = json.load(json_file)

In [ ]:
# path for saving the results
file_path = f'llm_responses/zero_shot_cot.json'
os.makedirs(os.path.dirname(file_path), exist_ok=True)

number_of_trials = 10 # adjust as needed
trial_collection = []

for i in range (0, number_of_trials):
    print(f"Run number {i + 1}")
    output_collection = []
    for i, argument in enumerate(tqdm(data.get('english_text'), desc='Processing arguments', unit = 'argument')):

        # zero-shot prompt
        zero_shot_prompt = [
            {'role': 'system', 'content': f'''You are an expert in identifying arguments and their premises in a text. An argument is a set of claims in which one or more of them—the premises—are put forward so as to offer reasons for another claim, the conclusion. A text that does not provide support or reasons for a conclusion is not an argument.'''},
            {'role': 'user', 'content': f'''You know that the text is related to this conclusion: {policy_option_list[data.get('question_id')[i] - 1]}. Can you extract the words in this sentence that state the premise or premises of this text: {argument} or extract "None" if the text is not an argument.

            Return the following JSON with the premises found in the text:
            {{
                "premise_1": "",
                ...
            }}. '''}
        ]

        # zero-shot chain-of-thought prompt
        zero_shot_cot_prompt = [
            {'role': 'system', 'content': '''You are an expert in identifying arguments and their premises in a text. An argument is a set of claims in which one or more of them—the premises—are put forward so as to offer reasons for another claim, the conclusion. A text that does not provide support or reasons for a conclusion is not an argument.'''},
            {'role': 'user', 'content': f'''Let's think step by step. You know that the text is related to this conclusion: {policy_option_list[data.get('question_id')[i] - 1]}. Can you extract the words in this sentence that state the premise or premises of this text: {argument} or extract "None" if the text is not an argument.
             Step 1, determine if the text provides any support or reasons for the conclusion. If it does not provide support or reasons for the conclusion, return {{"premise_1": "None"}} and don't execute the next steps.
             Step 2, If it does provide support or reasons, return the following JSON with the premises found in the text:
            {{
                "premise_1": "",
                ...
            }}.
             '''}]

        # adjust prompt message as needed
        output = llm_chat(model_name='llama2:latest',
                           message=zero_shot_cot_prompt)

        output_collection.append(output)
        print(output)
    trial_collection.append(output_collection)

# save results
with open(file_path, 'w') as json_file:
    json.dump(trial_collection, json_file)